# 1. Business Problem


Company XYZ is a worldwide e-commerce site with localized versions of the site.

A data scientist at XYZ noticed that Spain-based users have a much higher conversion rate than any other Spanish-speaking country. She therefore went and talked to the international team in charge of Spain And LatAm to see if they had any ideas about why that was happening.

Spain and LatAm country manager suggested that one reason could be translation. All Spanish- speaking countries had the same translation of the site which was written by a Spaniard. They agreed to try a test where each country would have its one translation written by a local. That is, Argentinian users would see a translation written by an Argentinian, Mexican users by a Mexican and so on. Obviously, nothing would change for users from Spain.

After they run the test however, they are really surprised cause the test is negative. I.e., it appears that the non-localized translation was doing better!

You are asked to:

1.Confirm that the test is actually negative. That is, it appears that the old version of the site with just one translation across Spain and LatAm performs better

2.Explain why that might be happening. Are the localized translations really worse?

# 2. Technical Question

To translate the business problem into a technical one, I want to define this issue in an A/B testing framework.
Here, the question is whether the localized translation performs better than the non-localized translation. The metrics is pretty much given - the conversion.

It is claimed that the test is negative. To validate this, we can conduct a 2-sample t-test on conversion rate while making sure the experiment design is valid.

# 3. Data Overview

In this challenge,I'll use Tableau first to do the preliminary data analysis and explore the patterns.

The dashboards below visualize the data distribution. Each grid pivots a feature and test/control. Blue represents number of records that didn't convert; orange represents those that converted. 

Percentage showing is out of the total population.


%%HTML
<div class='tableauPlaceholder' id='viz1580238654973' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Bo&#47;Book1_15802386341360&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Book1_15802386341360&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Bo&#47;Book1_15802386341360&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1580238654973');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1000px';vizElement.style.height='827px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [108]:
%%HTML
<div class='tableauPlaceholder' id='viz1580238654973' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Bo&#47;Book1_15802386341360&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Book1_15802386341360&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Bo&#47;Book1_15802386341360&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1580238654973');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1000px';vizElement.style.height='827px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [109]:
%%HTML
<div class='tableauPlaceholder' id='viz1580238803638' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;9J&#47;9JP36TZ9S&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;9JP36TZ9S' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;9J&#47;9JP36TZ9S&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1580238803638');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1000px';vizElement.style.height='827px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

From the visualizations above, we can see the experimenter didn't have a 50/50 split and was exposed 5 days. The distribution between the test and the control of some variants are not the same.
We can see the amount of people in the test group is smaller than the control group, and the percentage of people who actually were using Spanish website in the test group is smaller than that of the control group as well. This might confound the result of the test. We need to look at the relative change in converted population after we account for the difference in the distribution.


# 3. Statistical Analysis

For this challenge, I'll use 2 sample T test because I want to see if there is a significant difference in the conversion percentage between the test and the control.

95% CI, one side hypothesis testing with p1 = conversion rate in test, p2 = conversion rate in control.
H_0: p1 = p2
H_a: p1 < p2

In [99]:
import pandas as pd
import numpy as np

In [100]:
test_table = pd.read_csv("test_table.csv")
user_table = pd.read_csv("user_table.csv")

In [101]:
len(test_table['user_id'].unique()) - len(user_table['user_id'].unique()) 

454

In [95]:
# left join so test_table keeps all the information
# for 454 records, there will be no sex, age, country information
df = pd.merge(test_table, user_table, on="user_id",how="left")
df.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0,M,32.0,Spain
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1,M,21.0,Mexico
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0,M,34.0,Spain
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1,F,22.0,Mexico
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1,M,19.0,Mexico


In [96]:
test = df[df["test"]==1]
control = df[df["test"]==0]

In [97]:
p1 = test["conversion"].mean()
p2 = control["conversion"].mean()
n1 = len(test)
n2 = len(control)
p_hat = (p1*n1+p2*n2)/(n1+n2)

In [98]:
t = (p1-p2)/(math.sqrt((p_hat*(1-p_hat))*(1/n1+1/n2)))
t

-18.20833653382862

From the test statistics, the localized version is significantly worse than the non-localized one. But as we saw in section 3, the distribution for certain variants is not the same between test and control. For example, the percentage of people in each group that were actually using the Spanish browser in the test is less than the one in the control. Argentina has a lot more population in the test than in the control. Last but not least, Spain shouldn't be in the consideration since there was no change.
This means the a/b test wasn't designed well because we didn't control some variants and that led to invalid result.

Let's try control the browser language and country.

In [104]:
test = df[(df["test"]==1) & (df['browser_language'] == 'ES')]
control = df[(df["test"]==0) & (df['browser_language'] == 'ES') & (df['country']!= 'Spain')]

In [105]:
p1 = test["conversion"].mean()
p2 = control["conversion"].mean()
n1 = len(test)
n2 = len(control)
p_hat = (p1*n1+p2*n2)/(n1+n2)
t = (p1-p2)/(math.sqrt((p_hat*(1-p_hat))*(1/n1+1/n2)))
t

-7.0629272358250095

T is much smaller, but still significantly worse.

In [107]:
for c in df['country'].value_counts().index:
    print (c)
    if (c!="Spain"):
        test = df[(df["test"]==1) & (df['browser_language'] == 'ES') & (df['country'] == c)]
        control = df[(df["test"]==0) & (df['browser_language'] == 'ES') & (df['country'] == c)]
        p1 = test["conversion"].mean()
        p2 = control["conversion"].mean()
        n1 = len(test)
        n2 = len(control)
        p_hat = (p1*n1+p2*n2)/(n1+n2)
        t = (p1-p2)/(math.sqrt((p_hat*(1-p_hat))*(1/n1+1/n2)))
        print (t)


Mexico
0.7218861604412272
Colombia
-0.8936220465991221
Spain
Argentina
-0.8833823665500959
Peru
0.41162430899559765
Venezuela
-0.32982397026024507
Chile
0.4141108592658423
Ecuador
-0.42348204891748126
Guatemala
0.36086562261158467
Bolivia
-0.03223568336304132
Honduras
-0.31330459591361975
El Salvador
-1.0762007934940858
Paraguay
-0.032357967129160045
Nicaragua
0.09042327401203906
Costa Rica
0.25335351588924826
Uruguay
0.44226078188975043
Panama
0.5051278760535604


There is no significant difference for any country. From this test, we cannot reject the null hypothesis.

# 4. Conclusion

From the visualization, we see there are some flaws in the A/B test. The statistical analysis shows that there was a significant decrease in performance in the localized version, but after we controlled some confounding variants - browser language and country, there is no evidence that the localized version is better or worse. It's recommended to conduct another A/B test, with a 50/50 split that ensures similar distribution of each variants, for at least a week of exposure.